In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.compute as pc
import os
import gc

import numpy as np

from utils import compute_null_values
from utils import var_filter
import pandas as pd
from utils import extract_division_names
from utils import siren_filter


In [2]:
import importlib
import utils
importlib.reload(utils)


print(dir(utils))



['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'column_filter', 'compute_null_proportions', 'compute_null_values', 'compute_stats', 'count_nb_lines', 'count_target_list', 'count_target_value', 'extract_division_names', 'filter_null_rows', 'np', 'os', 'pa', 'pc', 'pq', 'show_columns', 'siren_filter', 'var_filter', 'year_simplified']


In [3]:
os.makedirs("datasets_temp", exist_ok=True)
os.makedirs("datasets_finaux", exist_ok=True)

L'utilisation de la mémoire tout au long du notebook étant très importante risquant de faire crasher le logiciel, on libère la mémoire allouée aux variables gérant les dataframes une fois qu'elles ne sont plus utilisées dans la suite.

In [4]:
#gestion mémoire  (l'équivalent de free en C)

gc.set_threshold(500, 5, 5)
gc.collect()

0

Préparation du bilan administratif (contient les labels, les dates de création et autres informations administratives)

In [5]:


from utils import show_columns

bilan_admin=pq.ParquetFile("datasets_originaux/StockUniteLegale_utf8.parquet")


#NOMS DES COLONNES
show_columns(bilan_admin)

Columns : ['siren', 'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale', 'dateCreationUniteLegale', 'sigleUniteLegale', 'sexeUniteLegale', 'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale', 'prenom4UniteLegale', 'prenomUsuelUniteLegale', 'pseudonymeUniteLegale', 'identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale', 'anneeEffectifsUniteLegale', 'dateDernierTraitementUniteLegale', 'nombrePeriodesUniteLegale', 'categorieEntreprise', 'anneeCategorieEntreprise', 'dateDebut', 'etatAdministratifUniteLegale', 'nomUniteLegale', 'nomUsageUniteLegale', 'denominationUniteLegale', 'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale', 'denominationUsuelle3UniteLegale', 'categorieJuridiqueUniteLegale', 'activitePrincipaleUniteLegale', 'nomenclatureActivitePrincipaleUniteLegale', 'nicSiegeUniteLegale', 'economieSocialeSolidaireUniteLegale', 'societeMissionUniteLegale', 'caractereEmployeurUniteLegale', 'activitePrincipaleNAF25UniteLegale']


In [6]:
# groupe=bilan_admin.read_row_group(0)

# df=groupe.to_pandas()

# df.head(10)
# df.columns

In [7]:
#from utils import count_target_value

#NOMBRE + PROPORTION D'ENTREPRISES SURVIVANTES

# nb_survivants,proportion_survivants=count_target_value("A", 'etatAdministratifUniteLegale',bilan_admin, proportion=True)
# print(f"Nombre de survivants : {nb_survivants}")
# print(f"Proportion de survivants : {proportion_survivants}")

In [8]:
#colonnes interessantes (on enlève les variables type prénom, nom, etc...)

columns=['siren', 
        'dateCreationUniteLegale',
        'etatAdministratifUniteLegale',
        'trancheEffectifsUniteLegale',
        'anneeEffectifsUniteLegale', 
        'categorieEntreprise',
        'anneeCategorieEntreprise',
        'categorieJuridiqueUniteLegale', 
        'activitePrincipaleUniteLegale', 
        'economieSocialeSolidaireUniteLegale',
        'societeMissionUniteLegale',
        'caractereEmployeurUniteLegale',
        'nombrePeriodesUniteLegale'
        ]


In [9]:




# null_values,null_values_proportion=compute_null_values(columns, bilan_admin, proportion=True)


# print("Proportion de valeurs null :")
# for i in range(len(null_values_proportion)):
#     print(f"    {columns[i]} : {null_values_proportion[i]}")




In [10]:
from utils import filter_null_rows

#FILTRE DES SECTEURS/CATEGORIES POUR NE GARDER QUE LES STARTUPS

categories_juridiques=["5710", "5532", "5505"]

secteurs_innovants= [
    # Numérique / Logiciel
    "58.21Z",  # Édition de jeux électroniques
    "58.29A",  # Édition de logiciels système et réseau
    "58.29B",  # Édition de logiciels applicatifs
    "62.01Z",  # Programmation informatique
    "62.02A",  # Conseil en systèmes et logiciels informatiques
    "62.02B",  # Tierce maintenance de systèmes et d’applications
    "62.09Z",  # Autres activités informatiques

    # Données / plateformes / cloud
    "63.11Z",  # Traitement de données, hébergement et activités connexes
    "63.12Z",  # Portails Internet
    "63.99Z",  # Autres services d’information n.c.a.

    # R&D / Deeptech
    "72.11Z",  # R&D en biotechnologie
    "72.19Z",  # R&D en autres sciences physiques et naturelles

    # Industrie technologique / hardware
    "26.11Z",  # Fabrication de composants électroniques
    "26.20Z",  # Fabrication d’ordinateurs et d’équipements périphériques
    "26.51A",  # Fabrication d’instruments de mesure

    # Conseil technologique à forte intensité innovation
    "70.22Z",  # Conseil pour les affaires et autres conseils de gestion (tech / SaaS)
]

var_filter(bilan_admin,'activitePrincipaleUniteLegale',"datasets_temp/bilan_admin_v2.parquet",secteurs_innovants)
bilan_admin=pq.ParquetFile("datasets_temp/bilan_admin_v2.parquet")

var_filter(bilan_admin,  'categorieJuridiqueUniteLegale',"datasets_temp/bilan_admin_v3.parquet",categories_juridiques)

#liberer memoire
gc.collect()

2

In [11]:
bilan_admin=pq.ParquetFile("datasets_temp/bilan_admin_v3.parquet")

In [12]:
df=pd.read_parquet("datasets_temp/bilan_admin_v3.parquet")
print(df.shape)
df.head(10)
df.columns

(284774, 35)


Index(['siren', 'statutDiffusionUniteLegale', 'unitePurgeeUniteLegale',
       'dateCreationUniteLegale', 'sigleUniteLegale', 'sexeUniteLegale',
       'prenom1UniteLegale', 'prenom2UniteLegale', 'prenom3UniteLegale',
       'prenom4UniteLegale', 'prenomUsuelUniteLegale', 'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale', 'trancheEffectifsUniteLegale',
       'anneeEffectifsUniteLegale', 'dateDernierTraitementUniteLegale',
       'nombrePeriodesUniteLegale', 'categorieEntreprise',
       'anneeCategorieEntreprise', 'dateDebut', 'etatAdministratifUniteLegale',
       'nomUniteLegale', 'nomUsageUniteLegale', 'denominationUniteLegale',
       'denominationUsuelle1UniteLegale', 'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale', 'categorieJuridiqueUniteLegale',
       'activitePrincipaleUniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale', 'nicSiegeUniteLegale',
       'economieSocialeSolidaireUniteLegale', 'societeMissionUniteLegale',

In [13]:
#SUPPRESSION DES COLONNES ININTERESSANTES
df=df.drop(['statutDiffusionUniteLegale', 
        'unitePurgeeUniteLegale',
        'sigleUniteLegale',
        'sexeUniteLegale',
       'prenom1UniteLegale',
        
         'prenom2UniteLegale',
           'prenom3UniteLegale',
       'prenom4UniteLegale', 
       'prenomUsuelUniteLegale',
         'pseudonymeUniteLegale',
       'identifiantAssociationUniteLegale',
        'dateDernierTraitementUniteLegale',
        'dateDebut',
        'nomUniteLegale',
          'nomUsageUniteLegale',
            'denominationUniteLegale',
       'denominationUsuelle1UniteLegale',
         'denominationUsuelle2UniteLegale',
       'denominationUsuelle3UniteLegale',
       'nomenclatureActivitePrincipaleUniteLegale',
         'nicSiegeUniteLegale',
         'caractereEmployeurUniteLegale',
         'activitePrincipaleNAF25UniteLegale'
       ], axis=1)

In [14]:
#SUPPRESSION DES LIGNES NULL POUR LE LABEL, SIREN ET DATE DE CREATION

df=df.dropna(subset=['siren'])
df = df.dropna(subset=['etatAdministratifUniteLegale'])
df = df.dropna(subset=['dateCreationUniteLegale'])


In [15]:
#OBSERVATION DES DIFFERENTES VALEURS PRISES PAR CERTAINES COLONNES

categorieEntreprise_vals=df["categorieEntreprise"].unique()
print(f"categorieEntreprise_vals : {categorieEntreprise_vals}")

trancheEffectifsUniteLegale=df["trancheEffectifsUniteLegale"].unique()
print(f"trancheEffectifsUniteLegale : {trancheEffectifsUniteLegale}")

anneeEffectifsUniteLegale=df["anneeEffectifsUniteLegale"].unique()
print(f"anneeEffectifsUniteLegale : {anneeEffectifsUniteLegale}")

anneeCategorieEntreprise=df["anneeCategorieEntreprise"].unique()
print(f"anneeCategorieEntreprise : {anneeCategorieEntreprise}")


economieSocialeSolidaireUniteLegale=df["economieSocialeSolidaireUniteLegale"].unique()
print(f"economieSocialeSolidaireUniteLegale : {economieSocialeSolidaireUniteLegale}")

societeMissionUniteLegale=df["societeMissionUniteLegale"].unique()
print(f"societeMissionUniteLegale : {societeMissionUniteLegale}")

etatAdministratifUniteLegale=df["etatAdministratifUniteLegale"].unique()
print(f"etatAdministratifUniteLegale : {etatAdministratifUniteLegale}")

nombre_periodes=df['nombrePeriodesUniteLegale'].unique()
print(f"nombrePeriodesUniteLegale : {nombre_periodes}")

categorieEntreprise_vals : ['PME' 'ETI' None 'GE']
trancheEffectifsUniteLegale : ['NN' '32' '12' '21' '02' '00' '01' '11' '03' '41' '22' '31' '42' '52'
 '51' '53']
anneeEffectifsUniteLegale : [  nan 2023.]
anneeCategorieEntreprise : [2023.   nan]
economieSocialeSolidaireUniteLegale : ['N' None 'O']
societeMissionUniteLegale : [None 'N' 'O']
etatAdministratifUniteLegale : ['A' 'C']
nombrePeriodesUniteLegale : [ 7  2  6  8  5  9 12  3 10  4 16 11 17 15 14 13 18 26 19 22  1]


In [16]:


dictNO={'N':-1, 'O':1}
dictcategorieentreprise={'ETI':2, 'GE':3, 'PME':1}
dictAC={'A':1 , 'C':0}

In [17]:
#MAPPING POUR N'AVOIR PLUS DE NULL

df['categorieEntreprise'] = df['categorieEntreprise'].map(dictcategorieentreprise).fillna(0).astype(int)
df['economieSocialeSolidaireUniteLegale'] = df['economieSocialeSolidaireUniteLegale'].map(dictNO).fillna(0).astype(int)
df['societeMissionUniteLegale'] = df['societeMissionUniteLegale'].map(dictNO).fillna(0).astype(int)
df['anneeEffectifsUniteLegale']=df['anneeEffectifsUniteLegale'].fillna(0).astype(int)
df['anneeCategorieEntreprise']=df['anneeCategorieEntreprise'].fillna(0).astype(int)
df['etatAdministratifUniteLegale']=df['etatAdministratifUniteLegale'].map(dictAC).astype(int)

df.head(20)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,005580436,1955-01-01,NN,0,7,1,2023,1,5710,70.22Z,-1,0
1,024074924,2002-02-28,NN,0,2,2,2023,1,5710,63.11Z,-1,0
2,035650167,1956-01-01,32,2023,6,2,2023,1,5710,26.11Z,0,0
3,036420040,1964-01-01,12,2023,8,1,2023,1,5710,72.19Z,-1,0
4,046750014,1967-01-01,12,2023,5,1,2023,1,5710,72.11Z,-1,0
5,055811095,1955-01-01,NN,0,7,0,0,0,5710,70.22Z,-1,0
6,057801136,1957-01-01,NN,0,5,0,0,0,5710,26.51A,0,0
7,062804414,1962-01-01,NN,0,9,1,2023,0,5710,70.22Z,-1,0
8,064502636,1964-01-01,NN,0,12,0,0,0,5710,62.02A,-1,0
9,067201087,1967-01-01,NN,0,7,2,2023,1,5710,70.22Z,-1,0


In [18]:
#CONVERSION DATE DE CREATION EN ANNEE DE CREATION


df['dateCreationUniteLegale'] = pd.to_datetime(df['dateCreationUniteLegale'], errors='coerce')
df['dateCreationUniteLegale'] = df['dateCreationUniteLegale'].dt.year.astype(int)

In [19]:
#CONVERSION DE TRANCHE EFFECTIF EN ENTIER

def tranche_to_int(val):
    if val == 'NN' or pd.isna(val):
        return -1
    else:
        return int(val)

df["trancheEffectifsUniteLegale"] = df['trancheEffectifsUniteLegale'].apply(tranche_to_int)
df["trancheEffectifsUniteLegale"].head(5)

0    -1
1    -1
2    32
3    12
4    12
Name: trancheEffectifsUniteLegale, dtype: int64

In [20]:
#IDEM POUR ACTIVITE PRINCIPALE (ON GARDE QUE LES 4 PREMIERS CHIFFRES)

def activite_to_int(val):
    return int(val[:2] + val[3:5])

df['activitePrincipaleUniteLegale'] = df['activitePrincipaleUniteLegale'].apply(activite_to_int)
df["siren"] = df["siren"].astype(int)

In [21]:
#VERIFICATION TYPE POUR N'AVOIR QUE DES ENTIERS
print(df.dtypes)
df.head(10)


siren                                  int64
dateCreationUniteLegale                int64
trancheEffectifsUniteLegale            int64
anneeEffectifsUniteLegale              int64
nombrePeriodesUniteLegale              int64
categorieEntreprise                    int64
anneeCategorieEntreprise               int64
etatAdministratifUniteLegale           int64
categorieJuridiqueUniteLegale          int64
activitePrincipaleUniteLegale          int64
economieSocialeSolidaireUniteLegale    int64
societeMissionUniteLegale              int64
dtype: object


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0


In [22]:
#RECHERCHE DE VALEURS NULL

df.isna().sum().sum()

np.int64(0)

In [23]:
#CONVERSION EN FICHIER PARQUET (SAUVEGARDE)

df.to_parquet("datasets_temp/bilan_admin_v4.parquet", engine='pyarrow', index=False)


#MEMOIRE LIBEREE
del df
gc.collect()

0

Gestion de fichier sur les données d'établissements (une entreprise peut avoir plusieurs établissements)

In [24]:
bilan_etablissement=pq.ParquetFile("datasets_originaux/StockEtablissement_utf8.parquet")


In [25]:
# table = bilan_etablissement.read_row_groups([0])  #lit le premier row group
# df = table.to_pandas().head(10)

# print(bilan_etablissement.schema.names)

# table = bilan_etablissement.read_row_group(0,columns=['siren','etatAdministratifEtablissement'])  # Lit le premier row group

# df = table.to_pandas().head(10)
# df

In [26]:
#FILTRER (ON NE GARDE QUE LES SIRENS PRESENTES DANS LE DATASET DES DONNEES ADMINISTRATIVES)
df_admin = pd.read_parquet("datasets_temp/bilan_admin_v4.parquet", columns=["siren"])
sirens_retenus = set(df_admin["siren"])


del df_admin
gc.collect()

writer = None

for i in range(bilan_etablissement.num_row_groups):
    batch = bilan_etablissement.read_row_group(i)

    batch = batch.set_column( batch.schema.get_field_index("siren"), "siren", pc.cast(batch["siren"], pa.int64()) ) 
    value_set = pa.array(list(sirens_retenus), type=pa.int64())
    mask = pc.is_in(batch["siren"], value_set=value_set) 
    batch_filtre = batch.filter(mask)

    if batch_filtre.num_rows > 0:
        if writer is None:
            writer = pq.ParquetWriter("datasets_temp/bilan_etablissement_v2.parquet", batch_filtre.schema)
        writer.write_table(batch_filtre)

    del batch, batch_filtre
    
    if (i + 1) % 20 == 0:
        gc.collect()

if writer:
    writer.close()


del sirens_retenus, writer
gc.collect()

0

In [27]:
bilan_etablissement=pq.ParquetFile("datasets_temp/bilan_etablissement_v2.parquet")

In [28]:

#ON CREE DE NOUVELLES VARIABLES (NOMBRE D'ETABLISSEMENTS ET PROPORTION D'ENTRE EUX QUI SONT OUVERTS)

dict_siren_nb_etablissements={}
dict_proportion_etablissement_ouvert={}


for i in range(bilan_etablissement.num_row_groups):
    groupe=bilan_etablissement.read_row_group(i,columns=["siren",'etatAdministratifEtablissement'])
    sirens = groupe.column("siren").to_pylist()
    
    etats=groupe.column('etatAdministratifEtablissement').to_pylist()
    for j in range(len(sirens)):
        if sirens[j] not in dict_siren_nb_etablissements:
            dict_siren_nb_etablissements[sirens[j]] = 0
            dict_proportion_etablissement_ouvert[sirens[j]] = 0

        dict_siren_nb_etablissements[sirens[j]] +=1
        if(etats[j]=='A'):
            dict_proportion_etablissement_ouvert[sirens[j]]+=1

    del groupe, sirens, etats
    
    if (i + 1) % 20 == 0:
        gc.collect()
for siren in dict_siren_nb_etablissements:
    dict_proportion_etablissement_ouvert[siren]/=dict_siren_nb_etablissements[siren]


gc.collect()

0

In [29]:
sirens = list(dict_siren_nb_etablissements.keys())
nb_etabs = [dict_siren_nb_etablissements[s] for s in sirens]
prop_ouverts = [dict_proportion_etablissement_ouvert[s] for s in sirens]

print(len(sirens))

284774


In [30]:

#ECRITURE DES NOUVELLES COLONNES

import gc

CHUNK_SIZE = 100_000  

output_file = "datasets_temp/bilan_etablissement_v3.parquet"
schema = pa.schema([
    ('siren', pa.int64()),
    ('nbEtablissements', pa.int32()),
    ('ProportionOuverts', pa.float32())
])

writer = pq.ParquetWriter(output_file, schema, compression='snappy')

# Traiter les clés par petits morceaux
all_sirens = list(dict_siren_nb_etablissements.keys())
total_sirens = len(all_sirens)



for start_idx in range(0, total_sirens, CHUNK_SIZE):
    end_idx = min(start_idx + CHUNK_SIZE, total_sirens)
   
    chunk = all_sirens[start_idx:end_idx]
    
    table = pa.Table.from_arrays(
        [
            chunk,
            [dict_siren_nb_etablissements[s] for s in chunk],
            [dict_proportion_etablissement_ouvert[s] for s in chunk]
        ],
        schema=schema
    )
    
    writer.write_table(table)
    
    if (start_idx // CHUNK_SIZE) % 10 == 0:
        gc.collect()
    
    if end_idx % 500_000 == 0:
        print(f"  {end_idx:,}/{total_sirens:,}")

writer.close()


del dict_siren_nb_etablissements, dict_proportion_etablissement_ouvert
del all_sirens, table, writer
gc.collect()


0

In [31]:
gc.collect()

df_etablissement=pd.read_parquet("datasets_temp/bilan_etablissement_v3.parquet")
df_admin=pd.read_parquet("datasets_temp/bilan_admin_v4.parquet")

In [32]:
siren_etab = set(df_etablissement["siren"].astype(str))
siren_admin = set(df_admin["siren"].astype(str))

print(siren_etab-siren_admin)

set()


In [33]:
df_admin.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0


In [34]:
df_etablissement.head(10)

,siren,nbEtablissements,ProportionOuverts
0,5580436,3,0.333333
1,24074924,1,1.000000
2,35650167,6,0.333333
3,36420040,16,0.125000
4,46750014,3,0.666667
5,55811095,3,0.000000
6,57801136,1,0.000000
7,62804414,5,0.000000
8,64502636,50,0.000000
9,67201087,2,0.500000


In [35]:
#FUSION ENTRE DATASET ADMINISTRATIF ET DATASET DES ETABLISSEMENTS

df_adminetab = df_admin.merge(df_etablissement, on='siren', how='left')


del df_admin, df_etablissement
gc.collect()




df_adminetab.shape

(284774, 14)

In [36]:
df_adminetab.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,ProportionOuverts
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0.333333
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,1.000000
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0.333333
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0.125000
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0.666667
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0.000000
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0.000000
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0.000000
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0.000000
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0.500000


In [37]:
df_adminetab.to_parquet('datasets_temp/bilan_adminetab.parquet',engine='pyarrow',compression='snappy',index=False)



Gestion du dataset sur les bilans financiers

In [38]:
bilan_financier=pq.ParquetFile("datasets_originaux/ratios_inpi_bce.parquet")

bilan_adminetab=pq.ParquetFile("datasets_temp/bilan_adminetab.parquet")



In [39]:

sirens_bilan_adminetab=extract_division_names(bilan_adminetab,"siren")
sirens_bilan_financier=extract_division_names(bilan_financier,"siren")



sirens_bilan_financier_list = sorted(sirens_bilan_financier) 
print(sirens_bilan_financier_list[:10])




sirens_bilan_adminetab_list = sorted(sirens_bilan_adminetab) 
print(sirens_bilan_adminetab_list[:10])

[5420120, 5450119, 5520176, 5520242, 5541552, 5580113, 5580501, 5580683, 5620034, 5620117]
[5580436, 24074924, 35650167, 36420040, 46750014, 55811095, 57801136, 62804414, 64502636, 67201087]


In [40]:
#FILTRER LES SIREN POUR N'AVOIR QUE LES SIRENS PRESENTES DANS LE DATASET ADMIN + ETABLISSEMENT

def sirens_absentes(sirens, dataset):

    missing_sirens = set(sirens)
    total_sirens = len(sirens)
    found_sirens = set()
    
    # Convertir en array PyArrow une seule fois au début
    sirens_array = pa.array(list(missing_sirens))
    
    
    
    for i in range(dataset.num_row_groups):
        group = dataset.read_row_group(i, columns=['siren'])
        
        # Trouver les SIREN de notre liste présents dans ce row group
        mask = pc.is_in(group['siren'], sirens_array)
        present_sirens_array = pc.filter(group['siren'], mask)
        present_sirens = set(present_sirens_array.to_pylist())
        found_sirens.update(present_sirens)
        
        # Affichage de progression
        if (i + 1) % 100 == 0 or (i + 1) == dataset.num_row_groups:
            progress = len(found_sirens) / total_sirens * 100 if total_sirens > 0 else 0
            
        
        # Arrêt anticipé si tous les SIREN sont trouvés
        if len(found_sirens) == total_sirens:
           
            break
    
    # Calculer les SIREN manquants
    missing_sirens = missing_sirens - found_sirens
    n_missing = len(missing_sirens)
    n_present = total_sirens - n_missing
    proportion_present = n_present / total_sirens if total_sirens > 0 else 0
    
    print(f"{n_missing} SIREN manquants sur {total_sirens} ({proportion_present:.2%} présents)\n")
    return missing_sirens


# Utilisation identique à votre code original
missing_sirens_bilan_financier = sirens_absentes(sirens_bilan_financier, bilan_adminetab)


1477497 SIREN manquants sur 1542770 (4.23% présents)



In [41]:
sirens_valides = set(sirens_bilan_financier) - set(missing_sirens_bilan_financier)

del sirens_bilan_adminetab, sirens_bilan_financier, missing_sirens_bilan_financier
gc.collect()


siren_filter(dataset=bilan_financier,column_name='siren',out_file_name="datasets_temp/bilan_financier_v2.parquet",list_siren=sirens_valides)


del sirens_valides
gc.collect()

0

In [42]:
#VERIFICATION
# 
#  bilan_financier=pq.ParquetFile("datasets_temp/bilan_financier_v2.parquet")

# sirens_bilan_financier=extract_division_names(bilan_financier,"siren")
# missing_sirens_bilan_financier = sirens_absentes(sirens_bilan_financier, bilan_admin)

In [43]:
df_financier = pd.read_parquet("datasets_temp/bilan_financier_v2.parquet")
print(df_financier.shape)
df_financier.columns

(219213, 23)


Index(['siren', 'date_cloture_exercice', 'Chiffre_d_affaires', 'Marge_brute',
       'EBE', 'EBIT', 'Resultat_net', 'Taux_d_endettement',
       'Ratio_de_liquidite', 'Ratio_de_vetuste', 'Autonomie_financiere',
       'Poids_BFR_exploitation_sur_CA', 'Couverture_des_interets',
       'CAF_sur_CA', 'Capacite_de_remboursement', 'Marge_EBE',
       'Resultat_courant_avant_impots_sur_CA',
       'Poids_BFR_exploitation_sur_CA_jours', 'Rotation_des_stocks_jours',
       'Credit_clients_Jours', 'Credit_fournisseurs_Jours', 'type_bilan',
       'confidentiality'],
      dtype='object')

In [44]:
df_financier.head(10)

,siren,date_cloture_exercice,Chiffre_d_affaires,Marge_brute,EBE,EBIT,Resultat_net,Taux_d_endettement,Ratio_de_liquidite,Ratio_de_vetuste,...,CAF_sur_CA,Capacite_de_remboursement,Marge_EBE,Resultat_courant_avant_impots_sur_CA,Poids_BFR_exploitation_sur_CA_jours,Rotation_des_stocks_jours,Credit_clients_Jours,Credit_fournisseurs_Jours,type_bilan,confidentiality
0,818842650,2019-12-31,0,0,-229,-229,-230,0.000,104.682,NaN,...,NaN,0.000,NaN,NaN,NaN,NaN,NaN,1482.969,S,Public
1,818845356,2022-12-31,0,0,-852,-1033,-1033,1.489,231.699,0.000,...,NaN,0.000,NaN,NaN,NaN,NaN,NaN,845.070,S,Public
2,818858086,2017-12-31,426247,426247,-5579,-7649,-8189,0.000,89.319,74.928,...,-1.269,0.000,-1.309,-1.963,-77.419,0.0,105.456,71.383,C,Public
3,818861965,2018-12-31,122080,117819,-9563,-12387,-12990,164.647,170.122,12.402,...,-8.317,-1.258,-7.833,-10.396,60.541,0.0,21.895,33.346,C,Public
4,818861965,2023-12-31,23320,23030,12014,2798,1767,3.778,284.314,NaN,...,47.097,0.186,51.518,11.998,866.316,0.0,1059.738,0.000,S,Public
5,818872707,2019-12-31,15447,15447,7519,7058,5999,26.420,1907.612,50.446,...,41.827,0.732,48.676,45.692,-11.979,0.0,0.000,25.253,C,Public
6,818905903,2023-12-31,0,0,0,0,15114,937.141,89.894,31.203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,Partiellement confidentiel
7,818917353,2017-12-31,0,0,-8349,-8832,0,1611.800,227.392,38.854,...,NaN,33.371,NaN,NaN,NaN,NaN,NaN,24.217,C,Public
8,818920076,2020-03-31,95267,95267,29978,29023,28466,2.184,970.001,16.483,...,29.936,0.127,30.573,29.554,82.806,0.0,102.149,41.617,C,Public
9,818964413,2022-12-31,3718438,3718438,177566,161863,121082,0.046,518.264,79.265,...,3.678,0.000,4.775,4.342,33.163,0.0,54.273,15.747,C,Public


In [45]:
#SUPPRESSION DES VARIABLES NON EXPLICATIVES

df_financier=df_financier.drop(["type_bilan","confidentiality"], axis=1)

In [46]:


df_financier['date_cloture_exercice'] = pd.to_datetime(df_financier['date_cloture_exercice'])
df_financier = df_financier.sort_values(['siren', 'date_cloture_exercice'])

In [47]:
#CONVERSION DATE BILAN EN ANNEE BILAN
df_financier['date_cloture_exercice'] = pd.to_datetime(df_financier['date_cloture_exercice'], errors='coerce')
df_financier['date_cloture_exercice'] = df_financier['date_cloture_exercice'].dt.year.astype(int)

In [48]:
#AJOUT DE LA COLONNE ANNEE DE CREATION DE L'ENTREPRISE

df_financier_avec_creation = df_financier.merge(
    df_adminetab[['siren', 'dateCreationUniteLegale','etatAdministratifUniteLegale']], 
    on='siren',
    how='left'  
)


In [49]:
#POURCENTAGE DE SURVIE

pourcentage = (df_financier_avec_creation['etatAdministratifUniteLegale'] == 1).mean() * 100
print(pourcentage)

82.65750662597566


In [50]:
#FILTRAGE DES LIGNES QUI CONCERNENT LES BILANS ENTRE LES ANNEES ANNEE DE CREATION ET ANNEE DE CREATION +2 

del df_financier
gc.collect()

df_filtre = df_financier_avec_creation[
    (df_financier_avec_creation['date_cloture_exercice'] >= df_financier_avec_creation['dateCreationUniteLegale']) &
    (df_financier_avec_creation['date_cloture_exercice'] <= df_financier_avec_creation['dateCreationUniteLegale'] + 2)
]


del df_financier_avec_creation
gc.collect()

0

In [51]:
# Vérification
df_filtre.head(20)
df_filtre.shape[0]
print(df_filtre["siren"].nunique())
pourcentage = (df_filtre['etatAdministratifUniteLegale'] == 1).mean() * 100
print(pourcentage)

36323
76.76805143422354


In [52]:
#CONSTRUCTION DE LA VERSION 1 DU DATASET (ON COMPTE TOUTES LES ENTREPRISES ET ON MET UNE VARIABLE DE PRESENCE SUR LA DECLARATION D UN BILAN DANS LES 3 PREMIERES ANNEES)

sirens_filtre = set(df_filtre["siren"].dropna().astype(str))

df_version1=df_adminetab.copy()


df_version1["aDéclaréBilan(s)FinancierSur3Ans"] = (
    df_version1["siren"]
    .astype(str)
    .isin(sirens_filtre)
    .astype(int)
)

df_version1.to_parquet('datasets_finaux/dataset_sans_details_financiers.parquet',engine='pyarrow',compression='snappy',index=False)


df_version1.head(10)

del sirens_filtre
gc.collect()

0

In [53]:
df_version1.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,ProportionOuverts,aDéclaréBilan(s)FinancierSur3Ans
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0.333333,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,1.000000,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0.333333,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0.125000,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0.666667,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0.000000,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0.000000,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0.000000,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0.000000,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0.500000,0


In [54]:


df_adminetab.head(10)


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,ProportionOuverts
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0.333333
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,1.000000
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0.333333
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0.125000
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0.666667
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0.000000
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0.000000
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0.000000
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0.000000
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0.500000


In [55]:


df_avec_bilan = df_version1[df_version1["aDéclaréBilan(s)FinancierSur3Ans"] == 1]

print(f"Nombre d'entreprises avec bilan : {len(df_avec_bilan)}")
df_avec_bilan.head(10)

Nombre d'entreprises avec bilan : 36323


,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,ProportionOuverts,aDéclaréBilan(s)FinancierSur3Ans
12465,480086784,2015,-1,0,3,1,2023,1,5710,7022,0,0,1,1.000000,1
22940,514943323,2009,-1,0,3,1,2023,1,5710,7022,-1,0,3,0.333333,1
24468,519715593,2010,2,2023,2,1,2023,1,5710,6209,-1,0,2,0.500000,1
24771,520267881,2010,-1,0,2,1,2023,0,5710,7022,0,0,1,0.000000,1
26073,522761139,2010,21,2023,5,1,2023,1,5710,7022,-1,0,8,0.625000,1
26105,522807981,2010,12,2023,3,1,2023,1,5710,6399,-1,0,1,1.000000,1
26650,523617173,2010,1,2023,6,1,2023,1,5710,6201,-1,0,3,0.333333,1
27021,524157872,2010,-1,0,2,0,0,0,5710,7022,0,0,1,0.000000,1
27276,524607868,2010,-1,0,4,1,2023,1,5710,6201,-1,0,2,0.000000,1
27627,525268975,2010,-1,0,4,1,2023,0,5710,7022,-1,0,2,0.000000,1


In [56]:
# #CONSTRUCTION DE LA VERSION 2 (ON NE RETIENT QUE LES ENTREPRISES QUI ONT DECLARE AU MOINS UN BILAN DANS LES 3 PREMIERES ANNEES)



df_filtre.head(10)
df_filtre.shape

(56616, 23)

In [57]:

missing_data = pd.DataFrame({
    "valeurs_manquantes": df_filtre.isna().sum(),
    "pourcentage": df_filtre.isna().mean() * 100
})

# Trier par pourcentage décroissant
# missing_data_sorted = missing_data.sort_values(by="pourcentage", ascending=True)
# print(missing_data_sorted)

# nb_colonnes_99_plus = (missing_data_sorted['pourcentage'] > 99).sum()
# print("Nombre de colonnes avec >99% de valeurs manquantes :", nb_colonnes_99_plus)


pourcentage_manquant = df_filtre.isna().mean() * 100

# Médiane
mediane_pourcentage = pourcentage_manquant.median()
print("Médiane du pourcentage de valeurs manquantes par colonne :", mediane_pourcentage)

Médiane du pourcentage de valeurs manquantes par colonne : 4.636498516320475


In [58]:
pd.set_option('display.max_rows', None)

df_stats=df_filtre.copy()


pourcentage_vide = df_stats.isna().mean() * 100

# Trier par ordre décroissant : les colonnes les plus remplies en premier
classement_vide = pourcentage_vide.sort_values(ascending=False)


print("Classement des colonnes par taux de vide :")
print(classement_vide)




df_stats.head(10)



Classement des colonnes par taux de vide :
Ratio_de_vetuste                        48.480995
Poids_BFR_exploitation_sur_CA           34.440794
Poids_BFR_exploitation_sur_CA_jours     34.440794
Rotation_des_stocks_jours               34.440794
CAF_sur_CA                              34.075173
Resultat_courant_avant_impots_sur_CA    33.827893
Marge_EBE                               33.827893
Credit_clients_Jours                    32.135792
Capacite_de_remboursement               17.535679
Couverture_des_interets                 17.502120
Credit_fournisseurs_Jours               15.527413
Ratio_de_liquidite                       4.636499
Taux_d_endettement                       0.174862
Autonomie_financiere                     0.081249
siren                                    0.000000
Resultat_net                             0.000000
EBIT                                     0.000000
EBE                                      0.000000
Marge_brute                              0.000000
Chiffre

,siren,date_cloture_exercice,Chiffre_d_affaires,Marge_brute,EBE,EBIT,Resultat_net,Taux_d_endettement,Ratio_de_liquidite,Ratio_de_vetuste,...,CAF_sur_CA,Capacite_de_remboursement,Marge_EBE,Resultat_courant_avant_impots_sur_CA,Poids_BFR_exploitation_sur_CA_jours,Rotation_des_stocks_jours,Credit_clients_Jours,Credit_fournisseurs_Jours,dateCreationUniteLegale,etatAdministratifUniteLegale
41502,480086784,2017,16927,16927,-33605,-33875,-21922,12.470,521.272,0.000,...,-135.417,-0.690,-198.529,-200.124,-356.789,0.000,0.000,43.591,2015,1
67875,514943323,2011,111048,56174,8907,-314,-1838,1310.938,146.013,66.939,...,6.502,4.997,8.021,-1.655,20.359,22.369,0.000,33.286,2009,1
70795,519715593,2012,149998,81719,32267,27064,22987,0.348,101.926,87.976,...,18.801,0.007,21.512,18.043,30.022,5.602,39.735,42.464,2010,1
71311,520267881,2012,37682,37682,-3997,-3665,-6761,0.000,38.141,89.354,...,-17.467,0.000,-10.607,-9.726,-177.764,0.000,0.000,52.993,2010,0
73527,522761139,2012,2216615,2216615,498608,497102,338041,13.778,298.840,9.377,...,15.097,0.239,22.154,22.113,39.616,0.000,78.778,21.579,2010,1
73563,522807981,2012,289372,275091,30471,3902,2447,777.829,76.959,64.803,...,10.587,1.199,10.473,1.024,-124.735,37.858,70.764,152.765,2010,1
74416,523617173,2012,155322,88096,-143840,-174688,212944,83.397,83.785,51.904,...,156.960,0.119,-92.608,-124.930,-116.213,7.751,354.581,102.188,2010,1
75114,524157872,2012,39327,39327,52,45,54,0.000,829.539,NaN,...,0.137,0.000,0.132,0.114,-4.238,0.000,0.000,73.204,2010,0
75491,524607868,2012,74920,73069,-13609,314,314,-0.986,16.776,58.341,...,-18.167,0.000,-18.165,0.419,-137.619,0.000,23.357,0.813,2010,1
76055,525268975,2012,281024,95221,34381,33058,26871,93.984,239.818,62.015,...,10.033,1.891,12.234,11.375,98.342,104.706,3.013,70.758,2010,0


In [59]:
df=df_filtre.copy()



In [60]:

#Taux de croissance annuel moyen (TCAM)

def tcam(vals):
    """
    Calcule le TCAM entre première et dernière valeur.
    Retourne 0 si une seule valeur, NaN si problème.
    """
    # Enlever les NaN
    vals_clean = vals[~np.isnan(vals)]
    

    # Une seule valeur = pas de croissance
    if len(vals_clean) <= 1:
        return 0.0
    
    v_start = vals_clean[0]
    v_end = vals_clean[-1]
    n_periods = len(vals_clean) - 1
    
    # Cas où valeur initiale = 0
    if v_start == 0:
        return 0.0 if v_end == 0 else np.nan
    
    # Cas où signes opposés
    if v_start * v_end < 0:
        return np.nan
    
    # Calcul normal
    try:
        ratio = v_end / v_start
        if ratio <= 0:
            return np.nan
        return (ratio**(1/n_periods) - 1) * 100
    except:
        return np.nan




In [61]:

# Vos colonnes financières
colonnes_financieres = [c for c in df.columns 
                        if c not in ["siren", "date_cloture_exercice", 
                                     "dateCreationUniteLegale", "etatAdministratifUniteLegale"]]

# Calcul des stats
resultats = []

for siren, groupe in df.groupby("siren"):
    ligne = {"siren": siren}
    
    # Trier par date
    groupe = groupe.sort_values('date_cloture_exercice')
    
    for col in colonnes_financieres:
        # Prendre les 3 premières valeurs non-nulles
        vals = groupe[col].dropna().values[:3]
        
        # Si moins de 3 valeurs, compléter avec la dernière
        if len(vals) == 1:
            vals = np.array([vals[0], vals[0], vals[0]])
        elif len(vals) == 2:
            vals = np.array([vals[0], vals[1], vals[1]])
        elif len(vals) == 0:
            vals = np.array([np.nan, np.nan, np.nan])
 
        ligne[f"{col}_tcam"] = tcam(vals)

        if np.all(np.isnan(vals)): 
            ligne[f"{col}_moyen"] = np.nan
        else: 
            ligne[f"{col}_moyen"] = np.nanmean(vals) # Moyenne 
            
        if np.all(np.isnan(vals)): 
            ligne[f"{col}_std"] = np.nan 
        else: 
            ligne[f"{col}_std"] = np.nanstd(vals)# Écart-type 
    
    resultats.append(ligne)

df_final = pd.DataFrame(resultats)


In [62]:
df_final.head(10)
df_final.isna().sum()


siren                                             0
Chiffre_d_affaires_tcam                        2711
Chiffre_d_affaires_moyen                          0
Chiffre_d_affaires_std                            0
Marge_brute_tcam                               2855
Marge_brute_moyen                                 0
Marge_brute_std                                   0
EBE_tcam                                       4488
EBE_moyen                                         0
EBE_std                                           0
EBIT_tcam                                      4510
EBIT_moyen                                        0
EBIT_std                                          0
Resultat_net_tcam                              4810
Resultat_net_moyen                                0
Resultat_net_std                                  0
Taux_d_endettement_tcam                        4446
Taux_d_endettement_moyen                         54
Taux_d_endettement_std                           54
Ratio_de_liq

In [63]:
cols_to_fill = [col for col in df_final.columns 
                if col.endswith(('_moyen', '_std'))]

for col in cols_to_fill:
    df_final[col] = df_final[col].fillna(df_final[col].median())

# Pour les TCAM, peut-être utiliser 0 au lieu de la médiane
cols_tcam = [col for col in df_final.columns if col.endswith('_tcam')]
for col in cols_tcam:
    df_final[col] = df_final[col].fillna(0)

In [64]:
df_final.isna().sum()

siren                                         0
Chiffre_d_affaires_tcam                       0
Chiffre_d_affaires_moyen                      0
Chiffre_d_affaires_std                        0
Marge_brute_tcam                              0
Marge_brute_moyen                             0
Marge_brute_std                               0
EBE_tcam                                      0
EBE_moyen                                     0
EBE_std                                       0
EBIT_tcam                                     0
EBIT_moyen                                    0
EBIT_std                                      0
Resultat_net_tcam                             0
Resultat_net_moyen                            0
Resultat_net_std                              0
Taux_d_endettement_tcam                       0
Taux_d_endettement_moyen                      0
Taux_d_endettement_std                        0
Ratio_de_liquidite_tcam                       0
Ratio_de_liquidite_moyen                

In [65]:

# Ajouter le label
df_final_final = df_final.merge(
    df_adminetab,
    on='siren',
    how='left'
)


In [66]:

df_final_final.head(10)

,siren,Chiffre_d_affaires_tcam,Chiffre_d_affaires_moyen,Chiffre_d_affaires_std,Marge_brute_tcam,Marge_brute_moyen,Marge_brute_std,EBE_tcam,EBE_moyen,EBE_std,...,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,ProportionOuverts
0,480086784,0.0,16927.0,0.0,0.0,16927.0,0.0,0.0,-33605.0,0.0,...,3,1,2023,1,5710,7022,0,0,1,1.000000
1,514943323,0.0,111048.0,0.0,0.0,56174.0,0.0,0.0,8907.0,0.0,...,3,1,2023,1,5710,7022,-1,0,3,0.333333
2,519715593,0.0,149998.0,0.0,0.0,81719.0,0.0,0.0,32267.0,0.0,...,2,1,2023,1,5710,6209,-1,0,2,0.500000
3,520267881,0.0,37682.0,0.0,0.0,37682.0,0.0,0.0,-3997.0,0.0,...,2,1,2023,0,5710,7022,0,0,1,0.000000
4,522761139,0.0,2216615.0,0.0,0.0,2216615.0,0.0,0.0,498608.0,0.0,...,5,1,2023,1,5710,7022,-1,0,8,0.625000
5,522807981,0.0,289372.0,0.0,0.0,275091.0,0.0,0.0,30471.0,0.0,...,3,1,2023,1,5710,6399,-1,0,1,1.000000
6,523617173,0.0,155322.0,0.0,0.0,88096.0,0.0,0.0,-143840.0,0.0,...,6,1,2023,1,5710,6201,-1,0,3,0.333333
7,524157872,0.0,39327.0,0.0,0.0,39327.0,0.0,0.0,52.0,0.0,...,2,0,0,0,5710,7022,0,0,1,0.000000
8,524607868,0.0,74920.0,0.0,0.0,73069.0,0.0,0.0,-13609.0,0.0,...,4,1,2023,1,5710,6201,-1,0,2,0.000000
9,525268975,0.0,281024.0,0.0,0.0,95221.0,0.0,0.0,34381.0,0.0,...,4,1,2023,0,5710,7022,-1,0,2,0.000000


In [67]:
#REARRANGER LES COLONNES


cols_admin = df_adminetab.columns.tolist() 
cols_final = [c for c in df_final_final.columns if c not in cols_admin]
df_final_final = df_final_final[cols_admin + cols_final]

In [68]:
df_final_final.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,...,Poids_BFR_exploitation_sur_CA_jours_std,Rotation_des_stocks_jours_tcam,Rotation_des_stocks_jours_moyen,Rotation_des_stocks_jours_std,Credit_clients_Jours_tcam,Credit_clients_Jours_moyen,Credit_clients_Jours_std,Credit_fournisseurs_Jours_tcam,Credit_fournisseurs_Jours_moyen,Credit_fournisseurs_Jours_std
0,480086784,2015,-1,0,3,1,2023,1,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,43.591,0.000000e+00
1,514943323,2009,-1,0,3,1,2023,1,5710,7022,...,0.000000e+00,0.0,22.369,0.0,0.0,0.000,0.000000e+00,0.0,33.286,0.000000e+00
2,519715593,2010,2,2023,2,1,2023,1,5710,6209,...,3.552714e-15,0.0,5.602,0.0,0.0,39.735,0.000000e+00,0.0,42.464,0.000000e+00
3,520267881,2010,-1,0,2,1,2023,0,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,52.993,0.000000e+00
4,522761139,2010,21,2023,5,1,2023,1,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,78.778,0.000000e+00,0.0,21.579,3.552714e-15
5,522807981,2010,12,2023,3,1,2023,1,5710,6399,...,0.000000e+00,0.0,37.858,0.0,0.0,70.764,0.000000e+00,0.0,152.765,0.000000e+00
6,523617173,2010,1,2023,6,1,2023,1,5710,6201,...,1.421085e-14,0.0,7.751,0.0,0.0,354.581,5.684342e-14,0.0,102.188,0.000000e+00
7,524157872,2010,-1,0,2,0,0,0,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,73.204,0.000000e+00
8,524607868,2010,-1,0,4,1,2023,1,5710,6201,...,0.000000e+00,0.0,0.000,0.0,0.0,23.357,0.000000e+00,0.0,0.813,1.110223e-16
9,525268975,2010,-1,0,4,1,2023,0,5710,7022,...,0.000000e+00,0.0,104.706,0.0,0.0,3.013,0.000000e+00,0.0,70.758,0.000000e+00


In [69]:
df_final_final.shape

(36323, 71)

In [70]:
df_final_final.isna().sum()

siren                                         0
dateCreationUniteLegale                       0
trancheEffectifsUniteLegale                   0
anneeEffectifsUniteLegale                     0
nombrePeriodesUniteLegale                     0
categorieEntreprise                           0
anneeCategorieEntreprise                      0
etatAdministratifUniteLegale                  0
categorieJuridiqueUniteLegale                 0
activitePrincipaleUniteLegale                 0
economieSocialeSolidaireUniteLegale           0
societeMissionUniteLegale                     0
nbEtablissements                              0
ProportionOuverts                             0
Chiffre_d_affaires_tcam                       0
Chiffre_d_affaires_moyen                      0
Chiffre_d_affaires_std                        0
Marge_brute_tcam                              0
Marge_brute_moyen                             0
Marge_brute_std                               0
EBE_tcam                                

In [71]:

# # Sauvegarder
df_final_final.to_parquet('datasets_finaux/dataset_avec_details_financiers.parquet',
                 engine='pyarrow', compression='snappy', index=False)


In [ ]:
fichiers=["datasets_temp/bilan_admin_v2.parquet",
          "datasets_temp/bilan_admin_v3.parquet",
          "datasets_temp/bilan_admin_v4.parquet",
          "datasets_temp/bilan_etablissement_v2.parquet",
          "datasets_temp/bilan_etablissement_v3.parquet",
          "datasets_temp/bilan_financier_v2.parquet",
          "datasets_temp/bilan_adminetab.parquet"]


for fichier in fichiers:
    os.remove(fichier)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets_temp/bilan_adminetab.parquet'

In [73]:

del df
del df_final_final
del df_stats
del df_filtre
del df_final

gc.collect()

1045